# Enunciado

Deseja-se comparar o tempo médio 
de espera para atendimento (min) em 2 hospitais. Para isso, coletou-se uma 
amostra com 20 pacientes em cada hospital. Verifique se há diferenças entre 
os tempos médios de espera nos dois hospitais. Considere α = 1%. 
Adicionalmente, calcule os intervalos de confiança para o tempo médio de 
espera nos dois hospitais. 

# Imports

In [1]:
import pandas as pd
from scipy.stats import f, ttest_ind, t
import math

# Leitura dos dados

In [2]:
alpha = 0.01

df = pd.read_excel('dados.xlsx')
df

,Hospital A,Hospital B
0,72,66
1,58,40
2,91,55
3,88,70
4,70,76
5,76,61
6,98,53
7,101,50
8,65,47
9,73,61


# Resolução

In [3]:
media_hospital_A = df['Hospital A'].mean()
media_hospital_B = df['Hospital B'].mean()

print(f"Media Hospital A: {media_hospital_A}")
print(f"Media Hospital B: {media_hospital_B}")

Media Hospital A: 81.5
Media Hospital B: 57.8


Aparentemente, a média de tempo de atendimento do hospital A é bem maior que do hospital B. Vamos ver se essa amostra tem relevância estatística.

 Vamos aplicar o teste T, que compara de médias em duas amostras independentes. Para a aplicação do teste T, é necessario verificar se as variancias são homogeneas, então teremos que fazer o teste F antes.

## Teste F

In [4]:
var_hospital_A = df['Hospital A'].var()
var_hospital_B = df['Hospital B'].var()

print(f"Variancia Hospital A: {var_hospital_A}")
print(f"Variancia Hospital B: {var_hospital_B}")

Variancia Hospital A: 137.73684210526315
Variancia Hospital B: 96.80000000000001


In [5]:
dados_hospital_a = df['Hospital A']
dados_hospital_b = df['Hospital B']

# Calcular a estatística F
# Por convenção, a maior variância vai no numerador
if var_hospital_A >= var_hospital_B:
    f_calculado = var_hospital_A / var_hospital_B
    gl_num = len(dados_hospital_a) - 1
    gl_den = len(dados_hospital_b) - 1
    print("Variância de A é maior ou igual à variância de B.")
else:
    f_calculado = var_hospital_B / var_hospital_A
    gl_num = len(dados_hospital_b) - 1
    gl_den = len(dados_hospital_a) - 1
    print("Variância de B é maior que a variância de A.")

print(f"Estatística F: {f_calculado}")
print(f"Graus de liberdade do numerador: {gl_num}")
print(f"Graus de liberdade do denominador: {gl_den}")

Variância de A é maior ou igual à variância de B.
Estatística F: 1.4229012614180077
Graus de liberdade do numerador: 19
Graus de liberdade do denominador: 19


In [6]:
p_value = f.sf(f_calculado, gl_num, gl_den) # sf = 1 - cdf
print(f"P-valor: {p_value}")

if p_value < alpha:
    print("Rejeitamos a hipótese nula de que as variâncias são iguais.")
else:
    print("Não rejeitamos a hipótese nula de que as variâncias são iguais.")

P-valor: 0.22460949688391565
Não rejeitamos a hipótese nula de que as variâncias são iguais.


Como não rejeitamos H0, então a variância das duas amostras é homogenêa

Como a variância é homogenea, iremos fazer o teste T segundo esse critério

## Teste T

In [7]:
# Aplicando o teste t para amostras independentes com variâncias iguais/homogeneas
t_calc, p_valor = ttest_ind(dados_hospital_a, dados_hospital_b, equal_var=True)

print(f"Estatística t: {t_calc}")
print(f"Valor-p: {p_valor}")

if p_valor < alpha:
    print("Rejeita H0")
else:
    print("Não rejeita H0")

Estatística t: 6.920824033665959
Valor-p: 3.147827649441996e-08
Rejeita H0


Rejeita-se a hipótese nula (H0). Isso significa que, ao nível de significância α utilizado, existe evidência estatística suficiente para concluir que há uma diferença significativa no tempo médio de espera entre o Hospital A e o Hospital B.

De fato, a diferença no tempo médio de espera entre os dois hospitais é estatisticamente significativa.

# Intervalo de confiança

In [11]:
def calcular_intervalo_confianca_t(media, desvio_padrao, t_critico, n):
    erro_padrao = desvio_padrao / math.sqrt(n)
    intervalo = (media - t_critico * erro_padrao, media + t_critico * erro_padrao)
    return intervalo

In [9]:
# --- Calculando o t crítico ---

# 1. Obtenha os tamanhos das amostras
n_a = len(dados_hospital_a)
n_b = len(dados_hospital_b)

# 2. Calcule os graus de liberdade (gl)
gl = n_a + n_b - 2

# 3. Calcule o t crítico
# Para um teste bilateral, há dois valores críticos: um positivo e um negativo.
# A função ppf(q, df) retorna o valor t tal que P(T <= t) = q.

# Para o valor t crítico positivo (que deixa alpha/2 na cauda superior):
t_critico_positivo = t.ppf(1 - alpha / 2, gl)

# Para o valor t crítico negativo (que deixa alpha/2 na cauda inferior):
t_critico_negativo = t.ppf(alpha / 2, gl)
# Alternativamente: t_critico_negativo = -t_critico_positivo

print(f"\n--- Cálculo do t crítico ---")
print(f"Nível de significância (alpha): {alpha}")
print(f"Graus de liberdade (gl): {gl}")
print(f"t crítico positivo: {t_critico_positivo:.3f}")
print(f"t crítico negativo: {t_critico_negativo:.3f}")


--- Cálculo do t crítico ---
Nível de significância (alpha): 0.01
Graus de liberdade (gl): 38
t crítico positivo: 2.712
t crítico negativo: -2.712


Para calcular o intervalo de confiança para cada hospital, **é necessário calcular o t critico INDIVIDUALMENTE**

In [ ]:
# Para o Hospital A
gl_a = n_a - 1
desvio_padrao_hospital_A = math.sqrt(var_hospital_A)
t_critico_hospital_a = t.ppf(1 - alpha / 2, gl_a)
ic_a = calcular_intervalo_confianca_t(media_hospital_A, desvio_padrao_hospital_A, t_critico_hospital_a, n_a)
print(f"Intervalo de confiança hospital A: {ic_a}")

# Para o Hospital B
gl_b = n_b - 1
desvio_padrao_hospital_B = math.sqrt(var_hospital_B)
t_critico_hospital_b = t.ppf(1 - alpha / 2, gl_b)
ic_b = calcular_intervalo_confianca_t(media_hospital_B, desvio_padrao_hospital_B, t_critico_hospital_b, n_b)
print(f"Intervalo de confiança hospital B: {ic_b}")

Intervalo de confiança hospital A: (73.99210839145069, 89.00789160854931)
Intervalo de confiança hospital B: (51.50594386581019, 64.09405613418981)
